In [1]:
import pandas as pd
import numpy as np
from os import listdir
import itertools
from IPython.display import clear_output

In [2]:
def GroupBy(listdir):
    pat = [(int(i.split('/')[-1].split('_')[0][3:]) * 1000) + (int(i.split('_')[1][3:])) for i in listdir]

    
    exercise = {'pat': pat,
          'dir': listdir}
    df = pd.DataFrame.from_dict(exercise)
    df = df.groupby('pat')['dir'].apply(list)
    return df

def GetUniquePatients(listdf):
    UniquePat = list(listdf[0].index)
    
    for df in listdf[1:]: # remove indexes which
        for index in UniquePat:
            if index not in df.index:
                UniquePat.remove(index)
    return UniquePat

In [4]:
# with supercleaned data
# Alpha = ['/datc/ortho/Cleaning/step3/alpha/' + i for i in listdir('/datc/ortho/Cleaning/step3/alpha/')]
# Bravo = ['/datc/ortho/Cleaning/step3/bravo/' + i for i in listdir('/datc/ortho/Cleaning/step3/bravo/')]
# Charlie = ['/datc/ortho/Cleaning/step3/charlie/' + i for i in listdir('/datc/ortho/Cleaning/step3/charlie/')]
# Delta = ['/datc/ortho/Cleaning/step3/delta/' + i for i in listdir('/datc/ortho/Cleaning/step3/delta/')]
# Echo = ['/datc/ortho/Cleaning/step3/echo/' + i for i in listdir('/datc/ortho/Cleaning/step3/echo/')]

# with normal cleaned data
Alpha = ['/datc/ortho/Cleaned oef/testset/alpha/' + i for i in listdir('/datc/ortho/Cleaned oef/testset/alpha/')]
Bravo = ['/datc/ortho/Cleaned oef/testset/bravo/' + i for i in listdir('/datc/ortho/Cleaned oef/testset/bravo/')]
Charlie = ['/datc/ortho/Cleaned oef/testset/charlie/' + i for i in listdir('/datc/ortho/Cleaned oef/testset/charlie/')]
Delta = ['/datc/ortho/Cleaned oef/testset/delta/' + i for i in listdir('/datc/ortho/Cleaned oef/testset/delta/')]
Echo = ['/datc/ortho/Cleaned oef/testset/echo/' + i for i in listdir('/datc/ortho/Cleaned oef/testset/echo/')]

In [5]:
dfA = GroupBy(Alpha)
dfB = GroupBy(Bravo)
dfC = GroupBy(Charlie)
dfD = GroupBy(Delta)
dfE = GroupBy(Echo)

In [6]:
Patients = GetUniquePatients([dfA, dfB, dfC, dfD, dfE])

# Create dataset

What is happening:
- Define the exercises
- loop over de patienten
- cross join de oefeninen voor maximaliseren van data
![](https://www.w3resource.com/w3r_images/cross-join-round.png)


In [7]:
def HeaderMaker(PatientFile):
    df_cleaned = pd.read_csv(PatientFile, header=None)

    df_cleaned = df_cleaned.rename(columns={0: "thorax_r_x", 1: "thorax_r_y", 2: "thorax_r_z"})
    df_cleaned = df_cleaned.rename(columns={3: "clavicula_r_x", 4: "clavicula_r_y", 5: "clavicula_r_z"})
    df_cleaned = df_cleaned.rename(columns={6: "scapula_r_x", 7: "scapula_r_y", 8: "scapula_r_z"})
    df_cleaned = df_cleaned.rename(columns={9: "humerus_r_x", 10: "humerus_r_y", 11: "humerus_r_z"})
    df_cleaned = df_cleaned.rename(columns={12: "ellebooghoek_r"})
    df_cleaned = df_cleaned.rename(columns={15: "thorax_l_x", 16: "thorax_l_y", 17: "thorax_l_z"})
    df_cleaned = df_cleaned.rename(columns={18: "clavicula_l_x", 19: "clavicula_l_y", 20: "clavicula_l_z"})
    df_cleaned = df_cleaned.rename(columns={21: "scapula_l_x", 22: "scapula_l_y", 23: "scapula_l_z"})
    df_cleaned = df_cleaned.rename(columns={24: "humerus_l_x", 25: "humerus_l_y", 26: "humerus_l_z"})
    df_cleaned = df_cleaned.rename(columns={27: "ellebooghoek_l"})
    
    return df_cleaned

def CalculateTurning(df, side):
    if side == 'r':
        column = 'humerus_r_x'
    elif side == 'l':
        column = 'humerus_l_x'
    
    turningcounter = 0
    growing = True
    previousi = 0
    for num, i in enumerate(df[column]):
        if i > previousi and growing == False:
            turningcounter += 1
            growing = True
        if i < previousi and growing:
            turningcounter += 1
            growing = False
        previousi = i
        if num == 1:
            turningcounter = 0
    return turningcounter

def CalculateOppervlakte(df, side):
    if side == 'r':
        x, y, z = ("humerus_r_x", "humerus_r_y", "humerus_r_z")
    elif side == 'l':
        x, y, z = ("humerus_l_x", "humerus_l_y", "humerus_l_z")
    
    

    xlist = df[x]
    ylist = df[y]
    zlist = df[z]


    index = 0
    NewList = []
    for num, ColumnList in enumerate([xlist, ylist, zlist]):
        templist = []
        for index in range(1, len(ColumnList)):
            try:
                vorige = ColumnList.iloc[index-1]
                volgende = ColumnList.iloc[index+1]
                templist.append(vorige - volgende)
            except(IndexError):
                templist.append(0)

        NewList.append({'x':list(ColumnList[1:]), 'y':templist})
    #             return NewList
    #             break

    OppervlakteLijst = []
    for item in NewList:
        TotaleOppervlakte = 0
        for index, CurrentXValue in enumerate(item['x']):
            if index == 0:
                continue

            PreviousXValue = item['x'][index - 1]

            if CurrentXValue > PreviousXValue:
                Multiplier = 1
            elif CurrentXValue < PreviousXValue:
                Multiplier = -1
            else:
                # Zelfde X waarde, dus geen oppervlakte
                continue

            LowestValue = min(item['y'])
            CurrentyValue = item['y'][index]
            PreviousyValue = item['y'][index - 1]

            # Vierkant oppervlakte berekenen
            breedte = abs(abs(CurrentXValue) - abs(PreviousXValue))
            hoogte = abs(LowestValue) + abs(min([CurrentyValue, PreviousyValue]))
            Oppervlakte = breedte * hoogte

            # Driehoek oppervlakte berekeken
            hoogteDrie = abs(CurrentyValue - PreviousyValue)
            Oppervlakte = Oppervlakte + 0.5 * hoogteDrie * breedte

            TotaleOppervlakte += Oppervlakte * Multiplier
        OppervlakteLijst.append(int(abs(TotaleOppervlakte)))
    return (OppervlakteLijst[0], OppervlakteLijst[1], OppervlakteLijst[2])

def CalculatePoints(df_cleaned, param):
    index = list(df_cleaned.index)
    length = len(index)


    firstindex = round(length*0.1)     # 10 percent
    secondindex = round(length*0.25)   # 25 percent
    thirdindex = round(length*0.5)     # 50 percent
    fourthindex = round(length*0.75)   # 75 percent
    fifthindex = round(length*0.9)     # 90 percent

    indexes = [index[firstindex], index[secondindex], index[thirdindex], index[fourthindex], index[fifthindex]]
    values = []
    for i in indexes:
        values.append(df_cleaned.loc[i][param])
    
    renamedvalue = []
    for num, i in enumerate(values):
        renamedvalue.append(i.rename(lambda x: x + '_%s' % (num))) # rename
    return renamedvalue

# Note
- Running all 5 excercises takes ~75 min
- running with 3 excercises takes a couple of minutes

In [8]:

exercises = [dfA, dfB, dfC, dfD, dfE]
data = {'pat': [],
       'cat': [],
       'turn': [],
       'EnergyX': [],
       'EnergyY': [],
       'EnergyZ': []}

param = ['thorax_r_x', 'thorax_r_y', 'thorax_r_z', 'clavicula_r_x', 'clavicula_r_y', 'clavicula_r_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', 'humerus_r_x', 'humerus_r_y', 'humerus_r_z', 'thorax_l_x', 'thorax_l_y', 'thorax_l_z', 'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z', 'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'humerus_l_x', 'humerus_l_y', 'humerus_l_z']

PatientDict = {}
for pat in Patients:
    exer = [i[pat] for i in exercises]
    CrossJoined = list(itertools.product(*exer)) # star means unpacking https://stackoverflow.com/questions/3941517/converting-list-to-args-when-calling-function
    for combo in list(CrossJoined):
        # combine all data from multiple files and add them to the df
        # filedictlist gets filled with all files
        filedictlist = []
        for file in combo: 
            # get all data from this file,
            # create a dict with all key:values
            filedict = {}
            df_cleaned = HeaderMaker(file)
            filedict['turnR'] = CalculateTurning(df_cleaned, 'r')
            filedict['turnL'] = CalculateTurning(df_cleaned, 'l')
            
            filedict['EnergyXR'], filedict['EnergyYR'], filedict['EnergyZR'] = CalculateOppervlakte(df_cleaned, 'r')
            filedict['EnergyXL'], filedict['EnergyYL'], filedict['EnergyZL'] = CalculateOppervlakte(df_cleaned, 'l')
            
            ValueList = CalculatePoints(df_cleaned, param)
            
            
            for i in ValueList: # add all points to the data
                for key in i.index:
                    filedict[key] = i[key]
            
            
            filedictlist.append(filedict)
        
        # rename all columns to the excersice
        # so there are no duplicates
        newdictlist = []
        for num, i in enumerate(filedictlist):
            if num == 0:
                add = 'Alpha_'
            elif num == 1:
                add = 'Bravo_'
            elif num == 2:
                add = 'Charlie_'
            elif num == 3:
                add = 'Delta_'
            elif num == 4:
                add = 'Echo_'
            else:
                add = 'err_'
            
            tmpdict = {}
            for key in i.keys():
                tmpdict[add+key] = i[key]
            newdictlist.append(tmpdict)
            
        # add the categorie
        newdictlist[0]['cat'] = int(pat/1000)
        newdictlist[0]['pat'] = pat
        
        
        # combine the different file dict to one big dict
        for num, i in enumerate(newdictlist):
            if num == 0:
                combodict = dict(i)
            else:
                combodict.update(i)
        
        # add the big dict to the overall data list
        for key, value in combodict.items():
            if key not in PatientDict.keys():
                PatientDict[key] = [value]
            else:
                PatientDict[key].append(value)
        length = len(PatientDict['Alpha_turnR'])
        if length % 100 == 0:
            clear_output()
            print(length)          
        
        
PatientDf = pd.DataFrame.from_dict(PatientDict)
PatientDf.to_csv('Data Alpha Bravo Charlie Delta Echo - Super cleaned testset.csv', sep=';', index=False)   

1700
